In [1]:
import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer

# EXAONE 모델과 토크나이저 로드
model = AutoModelForCausalLM.from_pretrained(
    "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct")

# CSV 파일 로드
input_csv_path = 'noise.csv'  # 입력 CSV 파일 경로
df = pd.read_csv(input_csv_path).head()

# 결과를 저장할 리스트
cleaned_texts = []

# 각 텍스트에 대해 노이즈 제거 작업 수행
for index, row in df.iterrows():
    noisy_text = row['text']  # 'text'는 노이즈가 포함된 컬럼 이름입니다.

    # 노이즈 제거를 위한 프롬프트 생성
    prompt = f"다음 텍스트에서 불필요한 특수문자나 잡음을 제거하고 정제된 문장으로 수정하세요: '{noisy_text}'"

    # 메시지 형식화
    messages = [
        {"role": "system", 
         "content": "You are EXAONE model from LG AI Research, a helpful assistant."},
        {"role": "user", "content": prompt}
    ]

    # 입력 텐서 준비
    input_ids = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    )

    # 모델에서 텍스트 생성 (노이즈 제거)
    output = model.generate(
        input_ids.to("cuda"),
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=128
    )

    # 출력 디코딩 및 정제된 텍스트 추출
    cleaned_text = tokenizer.decode(output[0], skip_special_tokens=True)

    # 결과 리스트에 추가
    cleaned_texts.append(cleaned_text)

# 정제된 텍스트를 데이터프레임에 추가
df['cleaned_text'] = cleaned_texts

# 결과 CSV 파일로 저장
output_csv_path = 'cleaned_texts.csv'
df.to_csv(output_csv_path, index=False)

print(f"노이즈가 제거된 텍스트가 {output_csv_path}에 저장되었습니다.")


2024-11-07 00:39:48.487491: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-07 00:39:48.501727: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730907588.526506  539460 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730907588.533555  539460 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-07 00:39:48.558473: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with t

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

노이즈가 제거된 텍스트가 cleaned_texts.csv에 저장되었습니다.


In [2]:
import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer

# EXAONE 모델과 토크나이저 로드
model = AutoModelForCausalLM.from_pretrained(
    "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct")

# CSV 파일 로드
input_csv_path = 'noise.csv'  # 입력 CSV 파일 경로
df = pd.read_csv(input_csv_path).head()

# 결과를 저장할 리스트
cleaned_texts = []

# 각 텍스트에 대해 노이즈 제거 작업 수행
for index, row in df.iterrows():
    noisy_text = row['text']  # 'text'는 노이즈가 포함된 컬럼 이름입니다.

    # 노이즈 제거를 위한 프롬프트 생성
    prompt = f"다음 텍스트에서 불필요한 특수문자나 잡음을 제거하고 정제된 문장으로 수정하세요: '{noisy_text}'"

    # 메시지 형식화
    messages = [
        {"role": "system", 
         "content": "You are EXAONE model from LG AI Research, a helpful assistant."},
        {"role": "user", "content": prompt}
    ]

    # 입력 텐서 준비
    input_ids = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    )

    # 모델에서 텍스트 생성 (노이즈 제거)
    output = model.generate(
        input_ids.to("cuda"),
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=128
    )

    # 출력 디코딩 및 정제된 텍스트 추출 (프롬프트 부분을 제외하고 실제 정제된 텍스트만 추출)
    cleaned_text = tokenizer.decode(output[0], skip_special_tokens=True)

    # 정제된 텍스트에서 프롬프트 부분을 제외한 텍스트만 추출
    start_idx = cleaned_text.find("다음은 불필요한 특수문자와 잡음을 제거한 정제된 문장입니다:")
    if start_idx != -1:
        cleaned_text = cleaned_text[start_idx + len("다음은 불필요한 특수문자와 잡음을 제거한 정제된 문장입니다:"):].strip()

    # 결과 리스트에 추가
    cleaned_texts.append(cleaned_text)

# 정제된 텍스트를 데이터프레임에 추가
df['cleaned_text'] = cleaned_texts

# 결과 CSV 파일로 저장
output_csv_path = 'cleaned_texts.csv'
df.to_csv(output_csv_path, index=False)

print(f"노이즈가 제거된 텍스트가 {output_csv_path}에 저장되었습니다.")


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

노이즈가 제거된 텍스트가 cleaned_texts.csv에 저장되었습니다.


In [ ]:
import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm  # 진행 상황을 시각적으로 표시하기 위한 라이브러리

# EXAONE 모델과 토크나이저 로드
model = AutoModelForCausalLM.from_pretrained(
    "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct")

# CSV 파일 로드
input_csv_path = 'noise.csv'  # 입력 CSV 파일 경로
df = pd.read_csv(input_csv_path)

# 결과를 저장할 리스트
cleaned_texts = []

# tqdm을 사용하여 진행 상황을 표시
for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing Texts"):
    noisy_text = row['text']  # 'text'는 노이즈가 포함된 컬럼 이름입니다.

    # 노이즈 제거를 위한 프롬프트 생성
    prompt = f"다음 텍스트에서 불필요한 특수문자나 잡음을 제거하고 정제된 문장으로 수정하세요: '{noisy_text}'"

    # 메시지 형식화
    messages = [
        {"role": "system", 
         "content": "You are EXAONE model from LG AI Research, a helpful assistant."},
        {"role": "user", "content": prompt}
    ]

    # 입력 텐서 준비
    input_ids = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    )

    # 모델에서 텍스트 생성 (노이즈 제거)
    output = model.generate(
        input_ids.to("cuda"),
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=128
    )

    # 출력 디코딩 및 정제된 텍스트 추출 (프롬프트 부분을 제외하고 실제 정제된 텍스트만 추출)
    cleaned_text = tokenizer.decode(output[0], skip_special_tokens=True)

    # 정제된 텍스트에서 프롬프트 부분을 제외한 텍스트만 추출
    start_idx = cleaned_text.find("다음은 불필요한 특수문자와 잡음을 제거한 정제된 문장입니다:")
    if start_idx != -1:
        cleaned_text = cleaned_text[start_idx + len("다음은 불필요한 특수문자와 잡음을 제거한 정제된 문장입니다:"):].strip()

    # 결과 리스트에 추가
    cleaned_texts.append(cleaned_text)

# 정제된 텍스트를 데이터프레임에 추가
df['cleaned_text'] = cleaned_texts

# 결과 CSV 파일로 저장
output_csv_path = 'cleaned_texts.csv'
df.to_csv(output_csv_path, index=False)

print(f"노이즈가 제거된 텍스트가 {output_csv_path}에 저장되었습니다.")


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
Processing Texts:   0%|          | 0/1595 [01:26<?, ?it/s]


In [ ]:
import torch
import pandas as pd
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer

# 모델과 토크나이저 로드
model = AutoModelForCausalLM.from_pretrained(
    "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct")

# CSV 파일 로드 (train.csv는 ID, text, target 열을 가진 파일)
df = pd.read_csv('noise.csv').head(5)

# 노이즈 제거 함수 정의
def denoise_text(text):
    # 시스템 메시지 및 사용자 메시지 구성
    messages = [
        {"role": "system", "content": "You are EXAONE model from LG AI Research, a helpful assistant."},
        {"role": "user", "content": f"Please clean this text: {text}"}
    ]
    
    # 텍스트를 토큰화하여 모델에 입력
    conversation = "\n".join([f"{message['role']}: {message['content']}" for message in messages])
    input_ids = tokenizer(conversation, return_tensors="pt").input_ids.to("cuda")

    # 모델로부터 출력 생성
    output = model.generate(
        input_ids,
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=128
    )

    # 생성된 텍스트 디코딩
    denoised_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return denoised_text

# 진행 상황 표시를 위한 tqdm 사용
tqdm.pandas(desc="Denoising text")

# 각 텍스트에 대해 노이즈 제거 수행 (진행 현황 표시)
df['denoised_text'] = df['text'].progress_apply(denoise_text)

# # 노이즈가 제거된 데이터셋을 새로운 CSV 파일로 저장
# df.to_csv('denoised_train.csv', index=False)

print("노이즈 제거 완료! denoised_train.csv 파일에 저장되었습니다.")

display(df)

In [ ]:
import torch
import pandas as pd
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer

# 모델과 토크나이저 로드
model = AutoModelForCausalLM.from_pretrained(
    "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct")

# CSV 파일 로드 (상위 10개 데이터만 사용)
df = pd.read_csv('noise.csv').head(10)

# 노이즈 제거 함수 정의
def denoise_text(text):
    messages = [
        {"role": "system", "content": "You are EXAONE model from LG AI Research, a helpful assistant."},
        {"role": "user", "content": f"Please clean this text: {text}"}
    ]
    
    conversation = "\n".join([f"{message['role']}: {message['content']}" for message in messages])
    input_ids = tokenizer(conversation, return_tensors="pt").input_ids.to("cuda")

    output = model.generate(
        input_ids,
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=128
    )

    # 모델 출력에서 "user"와 "system" 메시지를 제외하고, 노이즈 제거된 텍스트만 추출
    denoised_text = tokenizer.decode(output[0], skip_special_tokens=True)
    
    # 실제 노이즈 제거된 텍스트만 반환 (시작 부분을 제거하고 원하는 부분만 남기기)
    start_index = denoised_text.find('```\n') + 4  # 첫 번째 '```' 이후의 텍스트부터
    denoised_text = denoised_text[start_index:].strip()

    return denoised_text

# 진행 상황 표시를 위한 tqdm 사용
tqdm.pandas(desc="Denoising text")

# 각 텍스트에 대해 노이즈 제거 수행 (진행 현황 표시)
df['denoised_text'] = df['text'].progress_apply(denoise_text)

# 노이즈가 제거된 데이터셋을 새로운 CSV 파일로 저장
df.to_csv('denoised_train_sample.csv', index=False)

print("노이즈 제거 완료! denoised_train_sample.csv 파일에 저장되었습니다.")


In [ ]:
import torch
import pandas as pd
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer

# 모델과 토크나이저 로드
model = AutoModelForCausalLM.from_pretrained(
    "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct")

# CSV 파일 로드 (상위 10개 데이터만 사용)
df = pd.read_csv('noise.csv').head(10)

# 노이즈 제거 함수 정의
def denoise_text(text):
    messages = [
        {"role": "system", "content": "You are EXAONE model from LG AI Research, a helpful assistant."},
        {"role": "user", "content": f"Please clean this text: {text}"}
    ]
    
    conversation = "\n".join([f"{message['role']}: {message['content']}" for message in messages])
    input_ids = tokenizer(conversation, return_tensors="pt").input_ids.to("cuda")

    output = model.generate(
        input_ids,
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=128
    )

    # 모델 출력에서 불필요한 부분을 제외하고, 노이즈 제거된 텍스트만 추출
    denoised_text = tokenizer.decode(output[0], skip_special_tokens=True)
    
    # "system"과 "user" 역할을 제외하고 실제 노이즈 제거된 텍스트만 추출
    # 예시: "I apologize..." 같은 부분을 제거
    start_index = denoised_text.find("Please clean this text:") + len("Please clean this text:")  # 실제 텍스트 시작 위치 찾기
    denoised_text = denoised_text[start_index:].strip()

    return denoised_text

# 진행 상황 표시를 위한 tqdm 사용
tqdm.pandas(desc="Denoising text")

# 각 텍스트에 대해 노이즈 제거 수행 (진행 현황 표시)
df['denoised_text'] = df['text'].progress_apply(denoise_text)

# 노이즈가 제거된 데이터셋을 새로운 CSV 파일로 저장
df.to_csv('denoised_train_sample.csv', index=False)

print("노이즈 제거 완료! denoised_train_sample.csv 파일에 저장되었습니다.")


In [ ]:
print(df['denoised_text'])

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns

# 1. 데이터 로드
df = pd.read_csv("noise_free.csv")  # noise_free.csv 경로를 정확히 지정하세요
texts = df['text'].values  # 텍스트 데이터

# 2. KoBERT 모델과 토크나이저 불러오기
tokenizer = AutoTokenizer.from_pretrained("monologg/kobert")
model = AutoModel.from_pretrained("monologg/kobert")

# 3. 텍스트를 KoBERT로 임베딩
def embed_text(texts):
    embeddings = []
    for text in texts:
        inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128)
        with torch.no_grad():
            outputs = model(**inputs)
        # 평균값을 구하여 텍스트 임베딩으로 사용
        embedding = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
        embeddings.append(embedding)
    return embeddings

# 임베딩 생성
embeddings = embed_text(texts)

# 4. KMeans 클러스터링
num_clusters = 7  # 7개의 클러스터로 분류
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(embeddings)

# 5. 클러스터 예측 결과
df['cluster'] = kmeans.labels_

# 6. 각 클러스터의 주요 단어 출력 (임베딩을 직접 해석하기 어려운 점을 고려하여 생략)
# 대신 각 클러스터별로 제목을 출력하여 확인

for i in range(num_clusters):
    cluster_titles = df[df['cluster'] == i]['text'].tolist()
    print(f"Cluster {i}:")
    print("\n".join(cluster_titles[:5]))  # 각 클러스터의 상위 5개 뉴스 제목 출력
    print()

# 7. 클러스터링 결과 확인 (선택 사항)
plt.figure(figsize=(8, 6))
sns.countplot(x='cluster', data=df)
plt.title("Cluster Distribution")
plt.xlabel("Cluster")
plt.ylabel("Count")
plt.show()

# 8. 결과 파일로 저장 (선택 사항)
df.to_csv("clustered_noise_free.csv", index=False)